In [626]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv
import time
load_dotenv()

True

In [627]:
data = []
base_url = "https://facebook.com/"

In [ ]:
def get_website(url = base_url):
    chrome_options = webdriver.ChromeOptions()
    # Tat thong bao (alert) va mo full size cua so
    prefs = {
    "profile.default_content_setting_values.notifications": 2,    # 1: Cho phép, 2: Chặn
    "profile.default_content_setting_values.geolocation": 2,     # Chặn hỏi vị trí
    "profile.default_content_setting_values.media_stream": 2,    # Chặn hỏi Camera/Mic
    "profile.managed_default_content_settings.images": 1,        # 2 nếu bạn muốn tắt cả load ảnh để chạy nhanh hơn
    "credentials_enable_service": False,                         # Tắt hỏi lưu mật khẩu
    "profile.password_manager_enabled": False                    # Tắt trình quản lý mật khẩu
    }

    chrome_options.add_experimental_option("prefs",prefs)
    chrome_options.add_argument("--force-device-scale-factor=0.6")
    driver = webdriver.Chrome(options=chrome_options)
    driver.maximize_window()

    # Truy cập vào facebook.com và tìm form đăng nhập, thông tin từ .env
    driver.get(url)
    login_form = driver.find_element(By.CSS_SELECTOR, "form[data-testid='royal_login_form']")
    username = login_form.find_element(By.CSS_SELECTOR,"input[data-testid='royal-email']")
    password = login_form.find_element(By.CSS_SELECTOR,"input[data-testid='royal-pass']")
    your_username = os.getenv("USERRNAME")
    your_password = os.getenv("PASSWORD")
    username.send_keys(your_username)
    password.send_keys(your_password)
    
    # Kiếm nút đăng nhập để bấm
    login_btn = login_form.find_element(By.CSS_SELECTOR,"button[data-testid='royal-login-button']")
    login_btn.click()

    print("Hiện đang giải Captcha...")
    try:
        wait = WebDriverWait(driver,60)
        search_bar = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[aria-label='Tìm kiếm trên Facebook']")))
        print("Đã nhận diện được search_bar")
    except Exception:
        import traceback
        print("--- PHÁT HIỆN LỖI THẬT SỰ TẠI ĐÂY ---")
        traceback.print_exc() 
        print("------------------------------------")
    
    time.sleep(10)

    return driver       

In [629]:
test_driver = get_website()

Hiện đang giải Captcha...
Đã nhận diện được search_bar


In [630]:
# Nút để mở bình luận
def open_post(driver):
    try:
        comment_btn = driver.find_element(By.CSS_SELECTOR,"div[data-ad-rendering-role='comment_button']")
        # Phải click bằng js vì selenium đã bị chặn bởi các thủ thuật như hiển thị size 0x0 
        test_driver.execute_script("arguments[0].click();", comment_btn)
    except:
        print("Không thấy post để mở!")

In [639]:
open_post(test_driver)

In [631]:
# Nút để đóng bình luận
def close_post(driver):
    try:
        close_btn = driver.find_element(By.CSS_SELECTOR,"div[aria-label='Đóng']")
        driver.execute_script("arguments[0].click();",close_btn)
    except:
        print("Chưa có bài viết được mở ra!")

In [638]:
close_post(test_driver)

In [466]:
comment_btns = test_driver.find_elements(By.CSS_SELECTOR,"div[data-ad-rendering-role='comment_button']")
print(comment_btns)

[<selenium.webdriver.remote.webelement.WebElement (session="a345ed17ce42cdf1bd53cecd40944731", element="f.8CB8314FA47A7DAD76B7B88EA7D42073.d.18FE98FFCE72C1E8AC59AEEFBDACFE6F.e.377")>, <selenium.webdriver.remote.webelement.WebElement (session="a345ed17ce42cdf1bd53cecd40944731", element="f.8CB8314FA47A7DAD76B7B88EA7D42073.d.18FE98FFCE72C1E8AC59AEEFBDACFE6F.e.415")>, <selenium.webdriver.remote.webelement.WebElement (session="a345ed17ce42cdf1bd53cecd40944731", element="f.8CB8314FA47A7DAD76B7B88EA7D42073.d.18FE98FFCE72C1E8AC59AEEFBDACFE6F.e.416")>]


In [669]:
def clean_post(driver):
    delete_script = """
        // Tìm các vùng nội dung chính dựa trên thuộc tính data-ad-rendering-role
        const postElements = document.querySelectorAll('[data-ad-rendering-role="story_message"], [data-ad-rendering-role="post_message"]');
        postElements.forEach(el => el.remove());

        // 2. Tìm và xóa các ảnh bài viết to
        const images = document.querySelectorAll('div.x1y1aw1k img, .x1pdlv7q img');
        images.forEach(img => {
            // Chỉ xóa nếu là ảnh bài viết (thường có kích thước lớn)
            if(img.height > 100) {
                img.closest('div.x1n2onr6')?.remove();
            }
        });
    """
    try:
        driver.execute_script(delete_script)
        print("Done: Đã 'san phẳng' nội dung bài viết.")
    except Exception as e:
        print(f"Lỗi JS: {e}")
    #driver.execute_script("arguments[0].remove();", context)

In [672]:
clean_post(test_driver)

Done: Đã 'san phẳng' nội dung bài viết.


In [ ]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
# Tìm tất cả các bài viết
def scroll_main(driver,max_scroll=20):
    dataset = set()
    scroll_cnt = 0
    while(scroll_cnt < max_scroll):
        # messages = driver.find_elements(By.CSS_SELECTOR, "div[data-visualcompletion='ignore-dynamic']")
        # for msg in messages:
        #     post_cmt = msg.find_element(By.CSS_SELECTOR,"div[data-ad-rendering-role='comment_button']")
            
        driver.execute_script("window.scrollBy(0, 1000);")
        scroll_cnt+=1
        if(scroll_cnt % 5 == 0):
            time.sleep(2)
    #print(f"Tổng cộng đã quét được: {len(dataset)} bài viết.")

def scroll_post(driver, max_scroll = 20):
    dataset = set()
    scroll_cnt = 0
    container = driver.find_element(By.CSS_SELECTOR, "div[role='dialog'], .x1n2onr6")
    actions = ActionChains(driver)
    actions.move_to_element_with_offset(container, 50, 50).click().perform()
    while(scroll_cnt < max_scroll):
        actions.move_to_element(container).send_keys(Keys.PAGE_DOWN).perform()
        scroll_cnt+=1

In [ ]:
# Hiện các phản hồi Facebook
def display_respond(driver):
    target_xpath = "//span[contains(text(), 'phản hồi') and contains(text(), 'tất cả') and not(contains(text(), 'khác'))]" # Sử dụng xpath để lọc ra các span có text là phản hồi
    try:
        responses = driver.find_elements(By.XPATH, target_xpath)
        while(len(responses) > 0):
            for response in responses:
                driver.execute_script("arguments[0].click();",response)
                time.sleep(2)
            scroll_post(driver,20)
            try:
                responses = driver.find_elements(By.XPATH, target_xpath)
            except:
                print("Đã hết phản hồi!")
                responses = 0
    except:
        print("Không có phản hồi nào")

In [550]:
scroll_main(test_driver)

In [691]:
# Demo thử phần tự tìm bài viết và scroll xuống, mở hết các bình luận replied
open_post(test_driver)
time.sleep(3)
clean_post(test_driver)
time.sleep(3)
scroll_post(test_driver)
time.sleep(3)
display_respond(test_driver)

Done: Đã 'san phẳng' nội dung bài viết.


In [552]:
display_respond(test_driver)